In [1]:
from pathlib import Path
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from functools import reduce
import pandas as pd
import numpy as np

In [2]:
data_dir = Path().cwd().parent / "data"

In [3]:
contractions_map = {"you'd": 'you would', "he'd": 'he would', "she's": 'she is', "where'd": 'where did', "might've": 'might have', \
                "he'll": 'he will', "they'll": 'they will',  "mightn't": 'might not', "you'd've": 'you would have', "shan't": 'shall not', \
                "it'll": 'it will', "mayn't": 'may not', "couldn't": 'could not', "they'd": 'they would', "so've": 'so have', \
                "needn't've": 'need not have', "they'll've": 'they will have', "it's": 'it is', "haven't": 'have not', "didn't": 'did not', \
                "y'all'd": 'you all would', "needn't": 'need not', "who'll": 'who will', "wouldn't've": 'would not have', "when's": 'when is', \
                "will've": 'will have', "it'd've": 'it would have', "what'll": 'what will', "that'd've": 'that would have', \
                "y'all're": 'you all are', "let's": 'let us', "where've": 'where have', "o'clock": 'oclock', "when've": 'when have', \
                "what're": 'what are', "should've": 'should have', "you've": 'you have', "they're": 'they are', "aren't": 'are not', \
                "they've": 'they have', "it'd": 'it would', "i'll've": 'i will have', "they'd've": 'they would have', "you'll've": 'you will have', \
                "wouldn't": 'would not', "we'd": 'we would', "hadn't've": 'had not have', "weren't": 'were not', "i'd": 'i would', \
                "must've": 'must have', "what's": 'what is', "mustn't've": 'must not have', "what'll've": 'what will have', "ain't": 'aint', \
                "doesn't": 'does not', "we'll": 'we will', "i'd've": 'i would have', "we've": 'we have', "oughtn't": 'ought not', \
                "you're": 'you are', "who'll've": 'who will have', "shouldn't": 'should not', "can't've": 'cannot have', "i've": 'i have', \
                "couldn't've": 'could not have', "why've": 'why have', "what've": 'what have', "can't": 'cannot', "don't": 'do not', \
                "that'd": 'that would', "who's": 'who is', "would've": 'would have', "there'd": 'there would', "shouldn't've": 'should not have', \
                "y'all": 'you all', "mustn't": 'must not', "she'll": 'she will', "hadn't": 'had not', "won't've": 'will not have', \
                "why's": 'why is', "'cause": 'because', "wasn't": 'was not', "shan't've": 'shall not have', "ma'am": 'madam', "hasn't": 'has not', \
                "to've": 'to have', "how'll": 'how will', "oughtn't've": 'ought not have', "he'll've": 'he will have', "we'd've": 'we would have', \
                "won't": 'will not', "could've": 'could have', "isn't": 'is not', "she'll've": 'she will have', "we'll've": 'we will have', \
                "you'll": 'you will', "who've": 'who have', "there's": 'there is', "y'all've": 'you all have', "we're": 'we are', "i'll": 'i will', \
                "i'm": 'i am', "how's": 'how is', "she'd've": 'she would have', "sha'n't": 'shall not', "there'd've": 'there would have', \
                "he's": 'he is', "it'll've": 'it will have', "that's": 'that is', "y'all'd've": 'you all would have', "he'd've": 'he would have', \
                "how'd": 'how did', "where's": 'where is', "so's": 'so as', "she'd": 'she would', "mightn't've": 'might not have'}

procedural_words = ['member','members','president',
    'hon','parliament','house','ask','asked','asks','question','questioned','questions','bills','bill',
    'party','parties','mp','mps','sir','madam','mr','gentleman','gentlemen','lady','ladies',
    'speaker','chair','motion','motions','vote','votes','order','yes','deputy','secretary',
    'chairman','chairwoman',
    'america','usa','american','americans',
    'pursuant','supply','supplementary','please','friend','s',
    'clause','amendment','i','ii','iii','section','sections', 'colleague', 'colleagues']

stop_words = stopwords.words("english")

In [4]:
def clean_text(text): 
    text = reduce(lambda a, kv: a.replace(*kv), contractions_map.items(), text.lower())
    text = text.replace("\t"," ").replace("\n"," ").replace("\r"," ")
    text = text.translate(str.maketrans(string.punctuation, " "*len(string.punctuation)))
    
    tokens = word_tokenize(text) 
    
    tokens = [w for w in tokens if w not in stop_words and w not in procedural_words and len(w)>2 and w!=' ' and not w.isdigit()]
    
    return " ".join(tokens)


In [5]:
def extract_speaker(speakerid): 
    speakerid=str(speakerid)
    return "".join([*speakerid][3:8])

In [6]:
house_102_to_108 = pd.read_csv(data_dir / "house_102_to_108.txt", sep="\t")

In [7]:
house_102_to_108["speech"].map(lambda x: len(x.split())).mean()

199.87319811738882

In [8]:
house_102_to_108.groupby(["congress", "speakerid"]).size().groupby("congress").describe()

,count,mean,std,min,25%,50%,75%,max
congress,,,,,,,,
102,441.0,181.496599,219.021057,2.0,56.00,121.0,219.0,2775.0
103,440.0,172.536364,217.981554,1.0,50.00,109.0,207.5,2284.0
104,439.0,234.981777,239.843127,7.0,78.00,152.0,301.5,1715.0
105,443.0,166.067720,168.328675,1.0,55.00,115.0,219.5,1404.0
106,435.0,168.048276,159.060186,2.0,62.00,123.0,214.0,1299.0
107,438.0,123.488584,128.189089,3.0,46.25,85.0,154.0,906.0
108,437.0,140.322654,146.166327,1.0,49.00,92.0,189.0,1027.0


In [9]:
house_102_to_108["cleaned_speech"] = house_102_to_108["speech"].map(clean_text)
house_102_to_108["cleaned_speaker"] = house_102_to_108["speakerid"].map(extract_speaker)

In [10]:
house_102_to_108 = house_102_to_108[["speech_id", "cleaned_speech", "cleaned_speaker", "party", "congress"]]

In [11]:
house_102_to_108.to_csv(data_dir / "cleaned_house_102_to_108.txt", sep="\t", index=False)

In [12]:
fox_ids = pd.read_csv(data_dir / "fox_ids.csv")

In [19]:
fox_ids["unique_id"] = fox_ids["unique_id"].map(str)

In [20]:
fox_house_102_to_108 = house_102_to_108.merge(fox_ids, how="inner", left_on="cleaned_speaker", right_on="unique_id")

In [21]:
fox_house_102_to_108.to_csv(data_dir / "fox_house_102_to_108.txt", sep="\t", index=False)